In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import pandas as pd
from urllib.error import HTTPError
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:
df=pd.read_excel("visit jeju total.xlsx",na_filter=False)
del df['Unnamed: 0']
df['리뷰']=''

In [22]:
df

,콘텐츠ID,콘텐츠분류,제목,지번주소,도로명주소,위도,경도,평점,조회수,좋아요수,리뷰수,SNS공유수,인기점수,태그,사진,소개,리뷰
0,CNTS_000000000022002,관광지,아홉굿마을 낙천리 농어촌체험마을,한경면 낙천리 1916,한경면 낙수로 97,33.4996,126.531,,4500,2,0,1,9006,#체험관광 #아이 #사계절 #체험 #마을산책 #마을관광 #어린이 #어트랙션,https://api.cdn.visitjeju.net/photomng/imgpath...,주민들이 직접 만든 1000개의 의자가 있는 마을,
1,CNTS_000000000021985,관광지,제주오름승마랜드,조천읍 번영로 1734-15,조천읍 번영로 1734-15,33.449,126.714,,2910,0,0,4,5820,#승마 #액티비티 #커플 #맑음 #오름 #체험 #도보여행 #도보 #레저/체험 #어트랙션,https://api.cdn.visitjeju.net/photomng/imgpath...,영화속처럼 말타고 숲을 거닌다,
2,CONT_000000000501105,숙박,폴에이리조트,제주특별자치도서귀포시법환동978번지,제주특별자치도 서귀포시 월드컵로45번길 40,33.2444,126.511,,3075,0,0,0,6150,#휴식/치유 #숙소 #리조트 #공공와이파이존 #수영장 #정원,https://api.cdn.visitjeju.net/photomng/imgpath...,"서귀포의 중심에 자리잡고 있으며, 앞으로는 바다에 떠 있는 범섬과 뒤로는 한라산을 ...",
3,CNTS_000000000022556,관광지,하모씨워킹,제주특별자치도서귀포시대정읍최남단해안로 130,제주특별자치도서귀포시대정읍최남단해안로 130,33.2077,126.291,,9514,1,0,11,19031,#해수욕장 #액티비티 #친구 #커플 #맑음 #체험 #레저/체험 #해변 #물놀이 #수...,https://api.cdn.visitjeju.net/photomng/imgpath...,물 맑기로 유명한 제주의 하모해수욕장을 직접 걸어볼 수 있는 서비스를 제공한다.,
4,CONT_000000000500523,관광지,제주평화박물관(전쟁평화박물관)[휴관중],제주특별자치도 한경면 청수서5길 63(청수리),제주시 한경면 청수서5길 63,33.3056,126.249,,9677,2,0,19,19360,#실내 #미술/박물관 #아이 #비.눈 #가을 #실내관광지 #문화관광 #박물관 #어트랙션,https://api.cdn.visitjeju.net/photomng/imgpath...,일제강점기 '과거의 이야기'를 직접 체험하면서 평화의 소중함을 느낄 수 있는 곳,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,CNTS_200000000012075,관광지,헌마공신김만일기념관,서귀포시 남원읍 서성로 919,서귀포시 남원읍 서성로 919,,,0,142,1,0,0,287,#커플 #부모 #혼자 #친구 #아이 #맑음 #흐림 #비.눈 #경관/포토 #휴식/힐링...,https://api.cdn.visitjeju.net/photomng/imgpath...,임진왜란 당시 전쟁에 사용할 말 500마리를 헌납하면서 국난극복에 기여한 김만일(1...,
3625,CNTS_000000000022305,음식점,옐로우카페 (yellow cafe),색달동 2864-10,제주 서귀포시 중문관광로 90,33.2502,126.414,5,3454,0,1,1,6913,#바나나 #빙그레 #카페 #아이스크림 #쉐이크 #음식 #쿠키 #디저트 #빵 #아메리...,https://api.cdn.visitjeju.net/photomng/imgpath...,아련한 추억을 되살려주는 바나나맛 우유가 컨셉인 중문 이색 카페,
3626,CNTS_000000000022006,관광지,참살이마을 명도암 농어촌체험마을,봉개동 270-3,제주시 명림로 268-71,33.4616,126.607,5,3019,0,1,1,6043,#체험관광 #친구 #커플 #흐림 #사계절 #체험 #마을산책 #마을관광 #어트랙션,https://api.cdn.visitjeju.net/photomng/imgpath...,자연 그대로의 삶이 있는 참살이마을,
3627,CNTS_000000000022021,음식점,모슬포에서,대정읍 하모리 824-3,서귀포시 대정읍 영서중로13번길 9-9,33.2215,126.255,5,1569,0,1,0,3143,#녹두빈대떡 #보말톳메밀전 #흑돼지김치전 #깅이죽 #성게김밥 #음식,https://api.cdn.visitjeju.net/photomng/imgpath...,"모슬포 중앙시장 안, 맛과 청결을 중요시하는 '모슬포에서'",


In [23]:
path='C:/Users/qkral_2w1w9og/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver=webdriver.Chrome(path, options=options)

for i in range(0,len(df)):
    
    try:
        id=df.iloc[i,0]
        driver.get('https://www.visitjeju.net/kr/detail/view?contentsid='+str(id)+'#')
        num=int(driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/div[2]/ul/li[3]/p[2]').text)
        pagenum=num//5+2
        k=0
        # 만약 리뷰가 없으면
        if( num == 0 ): 
            raise Exception
            
        #  리뷰가 50의 배수
        elif(num%50==0):
            k = 0; num//=5;
            
        #else:
            #num//=5+1;
            
        driver.find_element(By.XPATH,'//*[@id="tab3"]/h2/a/span').click()
    except: continue
    try:
        for j in range(4,13):
        
            
            #driver.find_element(By.XPATH,'//*[@id="paging"]/a['+str(j)+']').send_keys(Keys.ENTER)

            df.at[i,'리뷰'] +=driver.find_element(By.XPATH,'//*[@id="contentsReviewItem0"]/div/div[2]/div[3]/p[1]').text; k+=1 
            if (num == k): raise Exception
            df.at[i,'리뷰'] +=driver.find_element(By.XPATH,'//*[@id="contentsReviewItem1"]/div/div[2]/div[3]/p[1]').text; k+=1 
            if (num == k): raise Exception
            df.at[i,'리뷰'] +=driver.find_element(By.XPATH,'//*[@id="contentsReviewItem2"]/div/div[2]/div[3]/p[1]').text; k+=1 
            if (num == k) :raise Exception
            df.at[i,'리뷰'] +=driver.find_element(By.XPATH,'//*[@id="contentsReviewItem3"]/div/div[2]/div[3]/p[1]').text; k+=1 
            if (num == k): raise Exception
            df.at[i,'리뷰'] +=driver.find_element(By.XPATH,'//*[@id="contentsReviewItem4"]/div/div[2]/div[3]/p[1]').text; k+=1 
            if (num == k) :raise Exception
            driver.find_element(By.XPATH,'//*[@id="paging"]/a['+str(j)+']').send_keys(Keys.ENTER)
                
        
            
    except: continue

C:\Users\qkral_2w1w9og\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [24]:
df.to_excel('reviewlast.xlsx')